In [1]:
!pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-z5ra0mcu
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-z5ra0mcu
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-py3-none-any.whl size=101064 sha256=195947ced32143e8550bf5e3f861c826929fae0dbe3fb314ebc381cb4875379d
  Stored in directory: /tmp/pip-ephem-wheel-cache-o_k_7a5y/wheels/bb/1f/f2/b57495012683b6b20bbae94a3915ec79753111452d79886abc
Successfully built keras-contrib


In [ ]:
# example of defining a 70x70 patchgan discriminator model
from keras.optimizers import Adam
from keras.initializers import RandomNormal
from keras.models import Model
from keras.models import Input
from keras.layers import Conv2D
from keras.layers import LeakyReLU
from keras.layers import Activation
from keras.layers import Concatenate
from keras.layers import BatchNormalization
from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization
from keras.utils.vis_utils import plot_model

# define the discriminator model
def define_discriminator(image_shape):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	# source image input
	in_image = Input(shape=image_shape)
	# C64
	d = Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(in_image)
	d = LeakyReLU(alpha=0.2)(d)
	# C128
	d = Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
	d = InstanceNormalization(axis=-1)(d)
	d = LeakyReLU(alpha=0.2)(d)
	# C256
	d = Conv2D(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
	d = InstanceNormalization(axis=-1)(d)
	d = LeakyReLU(alpha=0.2)(d)
	# C512
	d = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
	d = InstanceNormalization(axis=-1)(d)
	d = LeakyReLU(alpha=0.2)(d)
	# second last output layer
	d = Conv2D(512, (4,4), padding='same', kernel_initializer=init)(d)
	d = InstanceNormalization(axis=-1)(d)
	d = LeakyReLU(alpha=0.2)(d)
	# patch output
	patch_out = Conv2D(1, (4,4), padding='same', kernel_initializer=init)(d)
	# define model
	model = Model(in_image, patch_out)
	# compile model
	model.compile(loss='mse', optimizer=Adam(lr=0.0002, beta_1=0.5), loss_weights=[0.5])
	return model

# define image shape
image_shape = (256,256,3)
# create the model
model = define_discriminator(image_shape)
# summarize the model
model.summary()
# plot the model
plot_model(model, to_file='discriminator_model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
# generator a resnet block
def resnet_block(n_filters, input_layer):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	# first layer convolutional layer
	g = Conv2D(n_filters, (3,3), padding='same', kernel_initializer=init)(input_layer)
	g = InstanceNormalization(axis=-1)(g)
	g = Activation('relu')(g)
	# second convolutional layer
	g = Conv2D(n_filters, (3,3), padding='same', kernel_initializer=init)(g)
	g = InstanceNormalization(axis=-1)(g)
	# concatenate merge channel-wise with input layer
	g = Concatenate()([g, input_layer])
	return g

In [ ]:
# define the standalone generator model
def define_generator(image_shape=(256,256,3), n_resnet=9):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	# image input
	in_image = Input(shape=image_shape)
	# c7s1-64
	g = Conv2D(64, (7,7), padding='same', kernel_initializer=init)(in_image)
	g = InstanceNormalization(axis=-1)(g)
	g = Activation('relu')(g)
	# d128
	g = Conv2D(128, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(g)
	g = InstanceNormalization(axis=-1)(g)
	g = Activation('relu')(g)
	# d256
	g = Conv2D(256, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(g)
	g = InstanceNormalization(axis=-1)(g)
	g = Activation('relu')(g)
	# R256
	for _ in range(n_resnet):
		g = resnet_block(256, g)
	# u128
	g = Conv2DTranspose(128, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(g)
	g = InstanceNormalization(axis=-1)(g)
	g = Activation('relu')(g)
	# u64
	g = Conv2DTranspose(64, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(g)
	g = InstanceNormalization(axis=-1)(g)
	g = Activation('relu')(g)
	# c7s1-3
	g = Conv2D(3, (7,7), padding='same', kernel_initializer=init)(g)
	g = InstanceNormalization(axis=-1)(g)
	out_image = Activation('tanh')(g)
	# define model
	model = Model(in_image, out_image)
	return model

In [ ]:
# example of an encoder-decoder generator for the cyclegan
from keras.optimizers import Adam
from keras.models import Model
from keras.models import Input
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import Activation
from keras.initializers import RandomNormal
from keras.layers import Concatenate
from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization
from keras.utils.vis_utils import plot_model

# generator a resnet block
def resnet_block(n_filters, input_layer):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	# first layer convolutional layer
	g = Conv2D(n_filters, (3,3), padding='same', kernel_initializer=init)(input_layer)
	g = InstanceNormalization(axis=-1)(g)
	g = Activation('relu')(g)
	# second convolutional layer
	g = Conv2D(n_filters, (3,3), padding='same', kernel_initializer=init)(g)
	g = InstanceNormalization(axis=-1)(g)
	# concatenate merge channel-wise with input layer
	g = Concatenate()([g, input_layer])
	return g

# define the standalone generator model
def define_generator(image_shape=(256,256,3), n_resnet=9):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	# image input
	in_image = Input(shape=image_shape)
	# c7s1-64
	g = Conv2D(64, (7,7), padding='same', kernel_initializer=init)(in_image)
	g = InstanceNormalization(axis=-1)(g)
	g = Activation('relu')(g)
	# d128
	g = Conv2D(128, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(g)
	g = InstanceNormalization(axis=-1)(g)
	g = Activation('relu')(g)
	# d256
	g = Conv2D(256, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(g)
	g = InstanceNormalization(axis=-1)(g)
	g = Activation('relu')(g)
	# R256
	for _ in range(n_resnet):
		g = resnet_block(256, g)
	# u128
	g = Conv2DTranspose(128, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(g)
	g = InstanceNormalization(axis=-1)(g)
	g = Activation('relu')(g)
	# u64
	g = Conv2DTranspose(64, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(g)
	g = InstanceNormalization(axis=-1)(g)
	g = Activation('relu')(g)
	# c7s1-3
	g = Conv2D(3, (7,7), padding='same', kernel_initializer=init)(g)
	g = InstanceNormalization(axis=-1)(g)
	out_image = Activation('tanh')(g)
	# define model
	model = Model(in_image, out_image)
	return model

# create the model
model = define_generator()
# summarize the model
model.summary()
# plot the model
plot_model(model, to_file='generator_model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
...
# input shape
image_shape = (256,256,3)
# generator: A -> B
g_model_AtoB = define_generator(image_shape)
# generator: B -> A
g_model_BtoA = define_generator(image_shape)
# discriminator: A -> [real/fake]
d_model_A = define_discriminator(image_shape)
# discriminator: B -> [real/fake]
d_model_B = define_discriminator(image_shape)

In [ ]:
# composite: A -> B -> [real/fake, A]
c_model_AtoBtoA = define_composite_model(g_model_AtoB, d_model_B, g_model_BtoA, image_shape)
# composite: B -> A -> [real/fake, B]
c_model_BtoAtoB = define_composite_model(g_model_BtoA, d_model_A, g_model_AtoB, image_shape)

# lego2house implementation

In [2]:
from os import listdir
from numpy import asarray
from numpy import vstack
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from numpy import savez_compressed
import numpy as np
from numpy import load

In [3]:
import numpy as np
from random import random
from numpy import load
from numpy import zeros
from numpy import ones
from numpy import asarray
from numpy.random import randint
from keras.optimizers import Adam
from keras.initializers import RandomNormal
from keras.models import Model
from keras.models import Input
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Activation
from keras.layers import Concatenate
from matplotlib import pyplot

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

from tensorflow import Tensor
from tensorflow.keras.layers import Input, Conv2D, ReLU, BatchNormalization,\
									Add, AveragePooling2D, Flatten, Dense
from tensorflow.keras.models import Model
from keras.datasets import fashion_mnist
from keras.utils import to_categorical


In [4]:
# load and prepare training images
def load_real_samples(filename):
	# load the dataset
	data = load(filename)
	# unpack arrays
	X1, X2 = data['arr_0'], data['arr_1']
	# scale from [0,255] to [-1,1]
	X1 = (X1 - 127.5) / 127.5
	X2 = (X2 - 127.5) / 127.5
	return [X1, X2]

In [5]:
# load image data
dataset = load_real_samples('../input/lego2house/legotohouse.npz')
print('Loaded', dataset[0].shape, dataset[1].shape)
image_shape = dataset[0].shape[1:]
print(image_shape)

Loaded (1238, 256, 256, 3) (535, 256, 256, 3)
(256, 256, 3)


In [10]:
from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization

# generator a resnet block
def resnet_block(n_filters, input_layer):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	# first layer convolutional layer
	g = Conv2D(n_filters, (3,3), padding='same', kernel_initializer=init)(input_layer)
	g = InstanceNormalization(axis=-1)(g)
	g = Activation('relu')(g)
	# second convolutional layer
	g = Conv2D(n_filters, (3,3), padding='same', kernel_initializer=init)(g)
	g = InstanceNormalization(axis=-1)(g)
	# concatenate merge channel-wise with input layer
	g = Concatenate()([g, input_layer])
	return g



# define the standalone generator model
def define_generator(image_shape, n_resnet=9):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	# image input
	in_image = Input(shape=image_shape)
	# c7s1-64
	g = Conv2D(64, (7,7), padding='same', kernel_initializer=init)(in_image)
	g = InstanceNormalization(axis=-1)(g)
	g = Activation('relu')(g)
	# d128
	g = Conv2D(128, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(g)
	g = InstanceNormalization(axis=-1)(g)
	g = Activation('relu')(g)
	# d256
	g = Conv2D(256, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(g)
	g = InstanceNormalization(axis=-1)(g)
	g = Activation('relu')(g)
	# R256
	for _ in range(n_resnet):
		g = resnet_block(256, g)
	# u128
	g = Conv2DTranspose(128, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(g)
	g = InstanceNormalization(axis=-1)(g)
	g = Activation('relu')(g)
	# u64
	g = Conv2DTranspose(64, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(g)
	g = InstanceNormalization(axis=-1)(g)
	g = Activation('relu')(g)
	# c7s1-3
	g = Conv2D(3, (7,7), padding='same', kernel_initializer=init)(g)
	g = InstanceNormalization(axis=-1)(g)
	out_image = Activation('tanh')(g)
	# define model
	model = Model(in_image, out_image)
	return model

In [11]:
# generator: A -> B
g_model_AtoB = define_generator(image_shape)
# generator: B -> A
g_model_BtoA = define_generator(image_shape)

In [12]:
import sys
from keras.layers.normalization import BatchNormalization
import matplotlib.pyplot as plt

from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD


from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD, Adam
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers import Activation

In [13]:
# this discription not working, giving constant loss values, something fissy. 
# Need to look into it
def define_discriminator(image_shape):
    # vgg16
    init = RandomNormal(stddev=0.02)
    
    model = Sequential()
    model.add(Conv2D(64,(3,3),activation='relu',input_shape=image_shape,padding='same'))
    model.add(Conv2D(64,(3,3),activation='relu',padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(128,(3,3),activation='relu',padding='same'))
    model.add(Conv2D(128,(3,3),activation='relu',padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(256,(3,3),activation='relu',padding='same'))
    model.add(Conv2D(256,(3,3),activation='relu',padding='same'))
    model.add(Conv2D(256,(3,3),activation='relu',padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(512,(3,3),activation='relu',padding='same'))
    model.add(Conv2D(512,(3,3),activation='relu',padding='same'))
    model.add(Conv2D(512,(3,3),activation='relu',padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(512,(3,3),activation='relu',padding='same'))
    model.add(Conv2D(512,(3,3),activation='relu',padding='same'))
    model.add(Conv2D(512,(3,3),activation='relu',padding='same'))
    model.add(Conv2D(1, (4,4), padding='same', kernel_initializer=init))
#     outputs = Conv2D(1, (4,4), padding='same', kernet_initializer=init)(x)
    
#     model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
#     model.add(Flatten())
#     model.add(Dense(4096,activation='relu')) # act. fnc. non linear
#     model.add(Dense(4096,activation='relu'))
#     model.add(Dense(10))
#     model.add(Activation('softmax'))             # optim. Adam .. default n lr ... or sgd. 
    model.compile(loss='mse', optimizer=Adam(lr = 0.0002), loss_weights=[0.5])
    return model

In [ ]:
def define_discriminator(include_top=False, weights='imagenet',
          input_tensor=None, input_shape=None,
          pooling=None,
          classes=2):
    """Instantiates the VGG16 architecture.
    Optionally loads weights pre-trained
    on ImageNet. Note that when using TensorFlow,
    for best performance you should set
    `image_data_format="channels_last"` in your Keras config
    at ~/.keras/keras.json.
    The model and the weights are compatible with both
    TensorFlow and Theano. The data format
    convention used by the model is the one
    specified in your Keras config file.
    # Arguments
        include_top: whether to include the 3 fully-connected
            layers at the top of the network.
        weights: one of `None` (random initialization)
            or "imagenet" (pre-training on ImageNet).
        input_tensor: optional Keras tensor (i.e. output of `layers.Input()`)
            to use as image input for the model.
        input_shape: optional shape tuple, only to be specified
            if `include_top` is False (otherwise the input shape
            has to be `(224, 224, 3)` (with `channels_last` data format)
            or `(3, 224, 244)` (with `channels_first` data format).
            It should have exactly 3 inputs channels,
            and width and height should be no smaller than 48.
            E.g. `(200, 200, 3)` would be one valid value.
        pooling: Optional pooling mode for feature extraction
            when `include_top` is `False`.
            - `None` means that the output of the model will be
                the 4D tensor output of the
                last convolutional layer.
            - `avg` means that global average pooling
                will be applied to the output of the
                last convolutional layer, and thus
                the output of the model will be a 2D tensor.
            - `max` means that global max pooling will
                be applied.
        classes: optional number of classes to classify images
            into, only to be specified if `include_top` is True, and
            if no `weights` argument is specified.
    # Returns
        A Keras model instance.
    # Raises
        ValueError: in case of invalid argument for `weights`,
            or invalid input shape.
    """
    if weights not in {'imagenet', None}:
        raise ValueError('The `weights` argument should be either '
                         '`None` (random initialization) or `imagenet` '
                         '(pre-training on ImageNet).')

    if weights == 'imagenet' and include_top and classes != 1000:
        raise ValueError('If using `weights` as imagenet with `include_top`'
                         ' as true, `classes` should be 1000')
    # Determine proper input shape
    input_shape = _obtain_input_shape(input_shape,
                                      default_size=224,
                                      min_size=48,
                                      data_format=K.image_data_format(),
                                      include_top=include_top)

    if input_tensor is None:
        img_input = Input(shape=input_shape)
    else:
        if not K.is_keras_tensor(input_tensor):
            img_input = Input(tensor=input_tensor, shape=input_shape)
        else:
            img_input = input_tensor
    # Block 1
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    # Block 3
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    # Block 4
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    # Block 5
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)
    
    outputs = Conv2D(1, (4,4), padding='same', kernet_initializer=init)(x)
    
#     if include_top:
#         # Classification block
#         x = Flatten(name='flatten')(x)
#         x = Dense(4096, activation='relu', name='fc1')(x)
#         x = Dense(4096, activation='relu', name='fc2')(x)
#         x = Dense(classes, activation='softmax', name='predictions')(x)
#     else:
#         if pooling == 'avg':
#             x = GlobalAveragePooling2D()(x)
#         elif pooling == 'max':
#             x = GlobalMaxPooling2D()(x)

#     # Ensure that the model takes into account
#     # any potential predecessors of `input_tensor`.
#     if input_tensor is not None:
#         inputs = get_source_inputs(input_tensor)
#     else:
#         inputs = img_input
    # Create model.
    model = Model(inputs, outputs, name='vgg16')

    # load weights
    if weights == 'imagenet':
        if include_top:
            weights_path = get_file('vgg16_weights_tf_dim_ordering_tf_kernels.h5',
                                    WEIGHTS_PATH,
                                    cache_subdir='models')
        else:
            weights_path = get_file('vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5',
                                    WEIGHTS_PATH_NO_TOP,
                                    cache_subdir='models')
        model.load_weights(weights_path)
        if K.backend() == 'theano':
            layer_utils.convert_all_kernels_in_model(model)

        if K.image_data_format() == 'channels_first':
            if include_top:
                maxpool = model.get_layer(name='block5_pool')
                shape = maxpool.output_shape[1:]
                dense = model.get_layer(name='fc1')
                layer_utils.convert_dense_weights_data_format(dense, shape, 'channels_first')

            if K.backend() == 'tensorflow':
                warnings.warn('You are using the TensorFlow backend, yet you '
                              'are using the Theano '
                              'image data format convention '
                              '(`image_data_format="channels_first"`). '
                              'For best performance, set '
                              '`image_data_format="channels_last"` in '
                              'your Keras config '
                              'at ~/.keras/keras.json.')
    return model

In [14]:
# discriminator: A -> [real/fake]
d_model_A = define_discriminator(image_shape)
# discriminator: B -> [real/fake]
d_model_B = define_discriminator(image_shape)

In [16]:
# define a composite model for updating generators by adversarial and cycle loss
def define_composite_model(g_model_1, d_model, g_model_2, image_shape):
	# ensure the model we're updating is trainable
	g_model_1.trainable = True
	# mark discriminator as not trainable
	d_model.trainable = False
	# mark other generator model as not trainable
	g_model_2.trainable = False
	# discriminator element
	input_gen = Input(shape=image_shape)
	gen1_out = g_model_1(input_gen)
	output_d = d_model(gen1_out)
	# identity element
	input_id = Input(shape=image_shape)
	output_id = g_model_1(input_id)
	# forward cycle
	output_f = g_model_2(gen1_out)
	# backward cycle
	gen2_out = g_model_2(input_id)
	output_b = g_model_1(gen2_out)
	# define model graph
	model = Model([input_gen, input_id], [output_d, output_id, output_f, output_b])
	# define optimization algorithm configuration
	opt = Adam(lr=0.0002, beta_1=0.5)
	# compile model with weighting of least squares loss and L1 loss
	model.compile(loss=['mse', 'mae', 'mae', 'mae'], loss_weights=[1, 5, 10, 10], optimizer=opt)
	return model

In [17]:
# composite: A -> B -> [real/fake, A]
c_model_AtoB = define_composite_model(g_model_AtoB, d_model_B, g_model_BtoA, image_shape)
# composite: B -> A -> [real/fake, B]
c_model_BtoA = define_composite_model(g_model_BtoA, d_model_A, g_model_AtoB, image_shape)

In [18]:
# train cyclegan models
def train(d_model_A, d_model_B, g_model_AtoB, g_model_BtoA, c_model_AtoB, c_model_BtoA, dataset):
	# define properties of the training run
	n_epochs, n_batch, = 50, 1
	# determine the output square shape of the discriminator
	n_patch = d_model_A.output_shape[1]
	# unpack dataset
	trainA, trainB = dataset
	# prepare image pool for fakes
	poolA, poolB = list(), list()
	# calculate the number of batches per training epoch
	bat_per_epo = int(len(trainA) / n_batch)
	# calculate the number of training iterations
	n_steps = bat_per_epo * n_epochs
	# manually enumerate epochs
	for i in range(n_steps):
		# select a batch of real samples
		X_realA, y_realA = generate_real_samples(trainA, n_batch, n_patch)
		X_realB, y_realB = generate_real_samples(trainB, n_batch, n_patch)
		# generate a batch of fake samples
		X_fakeA, y_fakeA = generate_fake_samples(g_model_BtoA, X_realB, n_patch)
		X_fakeB, y_fakeB = generate_fake_samples(g_model_AtoB, X_realA, n_patch)
		# update fakes from pool
		X_fakeA = update_image_pool(poolA, X_fakeA)
		X_fakeB = update_image_pool(poolB, X_fakeB)
		# update generator B->A via adversarial and cycle loss
		g_loss2, _, _, _, _  = c_model_BtoA.train_on_batch([X_realB, X_realA], [y_realA, X_realA, X_realB, X_realA])
		# update discriminator for A -> [real/fake]
		dA_loss1 = d_model_A.train_on_batch(X_realA, y_realA)
		dA_loss2 = d_model_A.train_on_batch(X_fakeA, y_fakeA)
		# update generator A->B via adversarial and cycle loss
		g_loss1, _, _, _, _ = c_model_AtoB.train_on_batch([X_realA, X_realB], [y_realB, X_realB, X_realA, X_realB])
		# update discriminator for B -> [real/fake]
		dB_loss1 = d_model_B.train_on_batch(X_realB, y_realB)
		dB_loss2 = d_model_B.train_on_batch(X_fakeB, y_fakeB)
		# summarize performance
		print('>%d, dA[%.3f,%.3f] dB[%.3f,%.3f] g[%.3f,%.3f]' % (i+1, dA_loss1,dA_loss2, dB_loss1,dB_loss2, g_loss1,g_loss2))
		# evaluate the model performance every so often
		if (i+1) % (bat_per_epo * 1) == 0:
			# plot A->B translation
			summarize_performance(i, g_model_AtoB, trainA, 'AtoB')
			# plot B->A translation
			summarize_performance(i, g_model_BtoA, trainB, 'BtoA')
		if (i+1) % (bat_per_epo * 5) == 0:
			# save the models
			save_models(i, g_model_AtoB, g_model_BtoA)

In [19]:
# select a batch of random samples, returns images and target
def generate_real_samples(dataset, n_samples, patch_shape):
	# choose random instances
	ix = randint(0, dataset.shape[0], n_samples)
	# retrieve selected images
	X = dataset[ix]
	# generate 'real' class labels (1)
	y = ones((n_samples, patch_shape, patch_shape, 1))
	return X, y

# generate a batch of images, returns images and targets
def generate_fake_samples(g_model, dataset, patch_shape):
	# generate fake instance
	X = g_model.predict(dataset)
	# create 'fake' class labels (0)
	y = zeros((len(X), patch_shape, patch_shape, 1))
	return X, y

# save the generator models to file
def save_models(step, g_model_AtoB, g_model_BtoA):
	# save the first generator model
	filename1 = 'g_model_AtoB_%06d_30.h5' % (step+1)
	g_model_AtoB.save(filename1)
	# save the second generator model
	filename2 = 'g_model_BtoA_%06d_30.h5' % (step+1)
	g_model_BtoA.save(filename2)
	print('>Saved: %s and %s' % (filename1, filename2))
    

    # update image pool for fake images
def update_image_pool(pool, images, max_size=50):
	selected = list()
	for image in images:
		if len(pool) < max_size:
			# stock the pool
			pool.append(image)
			selected.append(image)
		elif random() < 0.5:
			# use image, but don't add it to the pool
			selected.append(image)
		else:
			# replace an existing image and use replaced image
			ix = randint(0, len(pool))
			selected.append(pool[ix])
			pool[ix] = image
	return asarray(selected)

# generate samples and save as a plot and save the model
def summarize_performance(step, g_model, trainX, name, n_samples=5):
	# select a sample of input images
	X_in, _ = generate_real_samples(trainX, n_samples, 0)
	# generate translated images
	X_out, _ = generate_fake_samples(g_model, X_in, 0)
	# scale all pixels from [-1,1] to [0,1]
	X_in = (X_in + 1) / 2.0
	X_out = (X_out + 1) / 2.0
	# plot real images
	for i in range(n_samples):
		pyplot.subplot(2, n_samples, 1 + i)
		pyplot.axis('off')
		pyplot.imshow(X_in[i])
	# plot translated image
	for i in range(n_samples):
		pyplot.subplot(2, n_samples, 1 + n_samples + i)
		pyplot.axis('off')
		pyplot.imshow(X_out[i])
	# save plot to file
	filename1 = '%s_generated_plot_%06d.png' % (name, (step+1))
	pyplot.savefig(filename1)
	pyplot.close()

# update image pool for fake images
def update_image_pool(pool, images, max_size=50):
	selected = list()
	for image in images:
		if len(pool) < max_size:
			# stock the pool
			pool.append(image)
			selected.append(image)
		elif random() < 0.5:
			# use image, but don't add it to the pool
			selected.append(image)
		else:
			# replace an existing image and use replaced image
			ix = randint(0, len(pool))
			selected.append(pool[ix])
			pool[ix] = image
	return asarray(selected)

In [20]:
train(d_model_A, d_model_B, g_model_AtoB, g_model_BtoA, c_model_AtoB, c_model_BtoA, dataset)

>1, dA[0.503,0.007] dB[0.501,0.013] g[16.858,19.484]
>2, dA[0.476,0.008] dB[0.485,0.001] g[15.148,16.695]
>3, dA[0.400,0.105] dB[0.449,0.027] g[11.791,12.441]
>4, dA[0.331,0.032] dB[0.378,0.058] g[11.121,13.748]
>5, dA[0.280,0.074] dB[0.234,0.210] g[10.028,11.236]
>6, dA[0.234,0.196] dB[0.237,0.073] g[8.827,8.988]
>7, dA[0.162,0.068] dB[0.296,0.031] g[13.284,14.551]
>8, dA[0.204,0.087] dB[0.292,0.065] g[8.627,9.505]
>9, dA[0.141,0.105] dB[0.196,0.193] g[7.364,8.183]
>10, dA[0.122,0.149] dB[0.147,0.095] g[9.475,10.095]
>11, dA[0.172,0.085] dB[0.261,0.045] g[9.313,10.313]
>12, dA[0.091,0.080] dB[0.250,0.041] g[9.554,10.094]
>13, dA[0.045,0.113] dB[0.230,0.079] g[8.956,10.031]
>14, dA[0.161,0.060] dB[0.146,0.233] g[7.927,10.159]
>15, dA[0.119,0.104] dB[0.124,0.083] g[6.830,7.467]
>16, dA[0.258,0.070] dB[0.222,0.042] g[8.206,10.204]
>17, dA[0.087,0.098] dB[0.233,0.089] g[8.590,9.321]
>18, dA[0.050,0.161] dB[0.176,0.146] g[6.171,5.645]
>19, dA[0.027,0.018] dB[0.080,0.264] g[9.629,11.500]
>2

>157, dA[0.153,0.108] dB[0.157,0.093] g[7.753,10.019]
>158, dA[0.146,0.111] dB[0.163,0.090] g[5.719,5.876]
>159, dA[0.141,0.115] dB[0.168,0.107] g[7.389,8.840]
>160, dA[0.130,0.126] dB[0.159,0.118] g[6.184,7.405]
>161, dA[0.105,0.125] dB[0.131,0.131] g[8.516,9.589]
>162, dA[0.094,0.138] dB[0.130,0.128] g[7.433,9.005]
>163, dA[0.135,0.112] dB[0.140,0.108] g[6.736,7.521]
>164, dA[0.184,0.082] dB[0.146,0.117] g[6.929,8.713]
>165, dA[0.178,0.086] dB[0.142,0.122] g[5.894,6.720]
>166, dA[0.175,0.095] dB[0.143,0.129] g[4.863,5.902]
>167, dA[0.165,0.103] dB[0.139,0.134] g[6.835,7.274]
>168, dA[0.157,0.115] dB[0.145,0.117] g[5.929,7.269]
>169, dA[0.142,0.121] dB[0.134,0.110] g[10.577,11.871]
>170, dA[0.133,0.122] dB[0.144,0.111] g[4.634,5.866]
>171, dA[0.052,0.172] dB[0.142,0.111] g[5.329,5.542]
>172, dA[0.082,0.106] dB[0.140,0.112] g[4.936,6.133]
>173, dA[0.161,0.094] dB[0.136,0.120] g[4.298,4.571]
>174, dA[0.166,0.089] dB[0.135,0.139] g[7.349,9.101]
>175, dA[0.172,0.088] dB[0.129,0.140] g[4.6

>312, dA[0.097,0.088] dB[0.055,0.134] g[6.908,6.857]
>313, dA[0.140,0.141] dB[0.097,0.069] g[5.049,5.564]
>314, dA[0.101,0.126] dB[0.100,0.250] g[4.710,5.283]
>315, dA[0.016,0.117] dB[0.156,0.110] g[6.990,8.352]
>316, dA[0.156,0.040] dB[0.127,0.105] g[7.424,7.219]
>317, dA[0.040,0.146] dB[0.157,0.091] g[7.728,8.797]
>318, dA[0.238,0.118] dB[0.155,0.099] g[4.708,5.334]
>319, dA[0.188,0.080] dB[0.153,0.069] g[4.366,5.109]
>320, dA[0.167,0.086] dB[0.159,0.061] g[5.753,6.300]
>321, dA[0.191,0.110] dB[0.200,0.135] g[5.507,6.844]
>322, dA[0.158,0.094] dB[0.132,0.104] g[5.347,5.827]
>323, dA[0.070,0.079] dB[0.115,0.101] g[6.204,6.602]
>324, dA[0.033,0.143] dB[0.123,0.094] g[7.208,8.650]
>325, dA[0.050,0.082] dB[0.120,0.081] g[7.498,8.221]
>326, dA[0.068,0.068] dB[0.165,0.101] g[7.800,9.355]
>327, dA[0.218,0.052] dB[0.109,0.173] g[6.608,7.551]
>328, dA[0.176,0.080] dB[0.127,0.072] g[4.895,5.945]
>329, dA[0.066,0.054] dB[0.137,0.080] g[8.649,7.318]
>330, dA[0.041,0.109] dB[0.103,0.066] g[8.561,

>467, dA[0.170,0.048] dB[0.101,0.074] g[8.546,8.185]
>468, dA[0.103,0.081] dB[0.139,0.008] g[6.586,7.520]
>469, dA[0.109,0.082] dB[0.132,0.014] g[5.230,5.892]
>470, dA[0.150,0.133] dB[0.092,0.040] g[7.576,7.685]
>471, dA[0.012,0.161] dB[0.068,0.035] g[6.374,7.235]
>472, dA[0.093,0.129] dB[0.029,0.260] g[3.794,4.640]
>473, dA[0.004,0.048] dB[0.083,0.047] g[6.166,7.203]
>474, dA[0.039,0.068] dB[0.059,0.097] g[5.603,6.729]
>475, dA[0.159,0.059] dB[0.189,0.189] g[5.701,6.435]
>476, dA[0.038,0.042] dB[0.093,0.169] g[5.973,7.356]
>477, dA[0.010,0.040] dB[0.036,0.043] g[7.706,9.243]
>478, dA[0.074,0.029] dB[0.122,0.042] g[6.455,7.718]
>479, dA[0.184,0.111] dB[0.079,0.006] g[6.459,6.175]
>480, dA[0.139,0.088] dB[0.242,0.035] g[6.682,7.338]
>481, dA[0.076,0.122] dB[0.163,0.084] g[6.716,7.594]
>482, dA[0.055,0.066] dB[0.295,0.039] g[7.540,7.431]
>483, dA[0.063,0.024] dB[0.202,0.029] g[7.767,7.966]
>484, dA[0.121,0.031] dB[0.094,0.018] g[5.409,6.077]
>485, dA[0.126,0.029] dB[0.190,0.022] g[7.693,

>622, dA[0.174,0.123] dB[0.228,0.066] g[4.889,5.393]
>623, dA[0.093,0.073] dB[0.162,0.081] g[5.639,6.332]
>624, dA[0.032,0.058] dB[0.156,0.111] g[6.373,6.911]
>625, dA[0.132,0.017] dB[0.122,0.149] g[6.346,7.758]
>626, dA[0.169,0.006] dB[0.092,0.158] g[5.864,6.871]
>627, dA[0.256,0.012] dB[0.103,0.145] g[4.200,5.359]
>628, dA[0.214,0.050] dB[0.096,0.122] g[3.858,4.216]
>629, dA[0.130,0.072] dB[0.061,0.113] g[9.402,10.483]
>630, dA[0.097,0.072] dB[0.043,0.105] g[7.029,8.184]
>631, dA[0.105,0.044] dB[0.067,0.103] g[5.915,6.625]
>632, dA[0.189,0.097] dB[0.150,0.157] g[5.543,8.245]
>633, dA[0.037,0.112] dB[0.096,0.171] g[7.639,9.463]
>634, dA[0.026,0.077] dB[0.097,0.189] g[6.668,8.845]
>635, dA[0.049,0.040] dB[0.038,0.075] g[6.772,7.718]
>636, dA[0.132,0.034] dB[0.178,0.122] g[4.896,5.949]
>637, dA[0.082,0.066] dB[0.224,0.039] g[6.044,6.262]
>638, dA[0.097,0.026] dB[0.157,0.054] g[7.426,9.259]
>639, dA[0.049,0.073] dB[0.153,0.074] g[6.298,8.098]
>640, dA[0.012,0.010] dB[0.110,0.051] g[7.447

>777, dA[0.106,0.026] dB[0.064,0.156] g[6.047,6.310]
>778, dA[0.071,0.043] dB[0.018,0.069] g[3.987,4.686]
>779, dA[0.119,0.062] dB[0.112,0.059] g[4.432,5.191]
>780, dA[0.058,0.048] dB[0.039,0.181] g[6.910,9.223]
>781, dA[0.211,0.045] dB[0.017,0.151] g[4.799,6.347]
>782, dA[0.017,0.053] dB[0.039,0.051] g[5.661,6.364]
>783, dA[0.191,0.092] dB[0.141,0.062] g[11.726,11.851]
>784, dA[0.011,0.122] dB[0.096,0.098] g[5.075,6.216]
>785, dA[0.007,0.040] dB[0.028,0.105] g[5.735,7.010]
>786, dA[0.094,0.085] dB[0.035,0.033] g[4.124,5.043]
>787, dA[0.018,0.045] dB[0.082,0.028] g[4.481,5.228]
>788, dA[0.167,0.064] dB[0.058,0.036] g[5.307,6.609]
>789, dA[0.078,0.028] dB[0.129,0.116] g[9.431,9.606]
>790, dA[0.023,0.071] dB[0.039,0.048] g[4.281,5.552]
>791, dA[0.010,0.121] dB[0.236,0.038] g[7.903,9.921]
>792, dA[0.097,0.010] dB[0.041,0.059] g[4.712,5.234]
>793, dA[0.085,0.023] dB[0.037,0.058] g[5.418,6.351]
>794, dA[0.022,0.030] dB[0.076,0.051] g[5.548,7.076]
>795, dA[0.021,0.015] dB[0.101,0.070] g[5.43

>932, dA[0.051,0.036] dB[0.170,0.025] g[5.798,5.639]
>933, dA[0.015,0.189] dB[0.124,0.024] g[7.037,7.697]
>934, dA[0.142,0.008] dB[0.048,0.021] g[4.577,5.972]
>935, dA[0.293,0.016] dB[0.085,0.061] g[5.814,6.372]
>936, dA[0.040,0.032] dB[0.019,0.042] g[6.706,6.221]
>937, dA[0.007,0.227] dB[0.025,0.012] g[8.093,9.592]
>938, dA[0.112,0.082] dB[0.015,0.081] g[4.228,4.624]
>939, dA[0.010,0.052] dB[0.181,0.003] g[7.017,7.421]
>940, dA[0.019,0.078] dB[0.023,0.015] g[8.479,10.005]
>941, dA[0.144,0.035] dB[0.057,0.075] g[3.966,4.736]
>942, dA[0.190,0.013] dB[0.013,0.050] g[6.071,7.575]
>943, dA[0.034,0.022] dB[0.084,0.171] g[6.999,7.843]
>944, dA[0.046,0.055] dB[0.055,0.056] g[6.154,6.885]
>945, dA[0.065,0.066] dB[0.132,0.031] g[5.032,5.134]
>946, dA[0.093,0.082] dB[0.255,0.137] g[3.606,4.248]
>947, dA[0.030,0.083] dB[0.059,0.217] g[4.613,6.148]
>948, dA[0.020,0.118] dB[0.088,0.013] g[6.857,6.850]
>949, dA[0.039,0.069] dB[0.060,0.027] g[5.417,5.500]
>950, dA[0.187,0.029] dB[0.134,0.003] g[6.984

>1085, dA[0.012,0.035] dB[0.134,0.007] g[7.071,9.219]
>1086, dA[0.285,0.020] dB[0.072,0.027] g[4.922,6.027]
>1087, dA[0.166,0.054] dB[0.122,0.173] g[3.713,3.855]
>1088, dA[0.154,0.072] dB[0.060,0.065] g[4.255,4.792]
>1089, dA[0.050,0.110] dB[0.079,0.192] g[4.346,4.936]
>1090, dA[0.065,0.136] dB[0.049,0.083] g[4.464,4.658]
>1091, dA[0.014,0.075] dB[0.089,0.094] g[4.376,5.042]
>1092, dA[0.012,0.065] dB[0.196,0.016] g[5.513,5.103]
>1093, dA[0.028,0.009] dB[0.133,0.011] g[7.654,7.815]
>1094, dA[0.082,0.038] dB[0.036,0.013] g[4.768,5.627]
>1095, dA[0.136,0.015] dB[0.058,0.070] g[4.085,4.115]
>1096, dA[0.019,0.016] dB[0.045,0.114] g[5.599,6.293]
>1097, dA[0.069,0.012] dB[0.079,0.022] g[4.877,5.205]
>1098, dA[0.296,0.027] dB[0.051,0.048] g[4.429,5.786]
>1099, dA[0.037,0.077] dB[0.063,0.021] g[4.102,4.779]
>1100, dA[0.015,0.124] dB[0.025,0.029] g[5.526,6.321]
>1101, dA[0.044,0.201] dB[0.046,0.017] g[5.864,5.428]
>1102, dA[0.108,0.040] dB[0.109,0.110] g[3.819,4.737]
>1103, dA[0.025,0.017] dB[0.

>1237, dA[0.079,0.202] dB[0.050,0.021] g[4.524,6.063]
>1238, dA[0.006,0.036] dB[0.021,0.233] g[6.052,6.514]


NameError: name 'summarize_performance' is not defined